# Restricted Boltzman Machine 
RBMs are shallow, two-layer neural nets that constitute the building blocks of deep-belief networks. The first layer of the RBM is called the visible, or input, layer, and the second is the hidden layer. The absense of an output layer is apparent. As we move forward we would learn why the output layer won't be needed.
<img src="figure1.png" width="150" height="50" title="Layers in RBM">
Figure1: Layers in RBM
Each circle in the figure above represents a neuron-like unit called a node, and nodes are simply where calculations take place. 
<img src="figure3.png" width="500" height="200" title="Layers in RBM">
Figure2: Structure of RBM
The nodes are connected to each other across layers, but no two nodes of the same layer are linked. That is, there is no intra-layer communication – this is the restriction in a restricted Boltzmann machine. Each node is a locus of computation that processes input, and begins by making stochastic decisions about whether to transmit that input or not. Each visible node takes a low-level feature from an item in the dataset to be learned.

Let's start by importing the packages that will be required in this project.

In [1]:
import sys
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
np_rng = np.random.RandomState(1234) #setting the random state

In [2]:
#Show all ouput
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(threshold=sys.maxsize)

#Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# graded
# import data
df = pd.read_excel('amazon.xlsx')

In [4]:
#run this and check if you have got the correct output
df.shape
df.head()

(1000, 1)

,Text
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In Topic Modelling, you find the best set of topics that describe the document. There are various ways to perform topic modelling one of which is RBM. You train your RBM on a set of documents. 
The visible layers will be the words in the text, the hidden layers will give the Topics. 
To input words into the visible layer, let's convert the trai data above into a bag of words model.

In [5]:
#graded
# create bag of words model 
#the final shape should be (number of documents, vocabulary)
# fit tf on the dataframe df
tf = CountVectorizer(stop_words='english',max_df=0.5) 
trainX = tf.fit_transform(df.Text)

trainX.shape
len(tf.vocabulary_)

(1000, 1642)

1642

>**Observation**: There are 1642 words in the vocabulary. 

In [6]:
# look at the dataframe
pd.DataFrame(trainX.toarray(), columns = tf.get_feature_names()).head()

,10,100,11,12,13,15,15g,18,20,2000,...,wouldn,wow,wrong,wrongly,year,years,yell,yes,z500a,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's verify that the vectorizer worked well taking 1st review as a sample

In [7]:
print(df.iloc[0].Text) #1st review
np.where(trainX.toarray()[0]==1)[0] #get indices in the vectorized review where the value is 1 i.e. word exists in vocabulary

So there is no way for me to plug it in here in the US unless I go by a converter.


array([ 326, 1081, 1524, 1582], dtype=int64)

In [8]:
#get words in vocabulary with same indices
tf.get_feature_names()[326]
tf.get_feature_names()[1081]
tf.get_feature_names()[1524]
tf.get_feature_names()[1582]

#check how many words are there in vocabulary for 1st sentence
print(sum(trainX.toarray()[1]))
trainX.shape

'converter'

'plug'

'unless'

'way'

4


(1000, 1642)

>**Observation**: In the bag-of-words vectorized data, the 1st review (document) has the words 'converter','plug','unless','way' set to 1 which is as expected as these words are part of the vocabulary.

Now that you have the bag of words model, let's define the number of visible and hidden units.

In [9]:
#graded
# define visible units
visibleUnits = trainX.shape[1] # vocabulary size ~1 line

# assign number of units
hiddenUnits = 5 # hyperparameter, this means that we are looking for 5 topics

In [10]:
visibleUnits

1642

In [11]:
#graded
# utility Functions

# deine the sigmoid function
def sigmoid(X):
    return 1/(1+np.exp(-X)) # ~ 1 line

## RBM as a Probabilistic Model
Restricted Boltzmann Machines are probabilistic. As opposed to assigning discrete values the model assigns probabilities. At each point in time the RBM is in a certain state. The state refers to the values of neurons in the visible and hidden layers v and h. The probability that a certain state of v and h can be observed is given by the following joint distribution:
<img src="figure4.png" width="200" height="70" title="Layers in RBM">

Eq. 2. Joint Distribution for v and h.
Here Z is called the ‘partition function’ that is the summation over all possible pairs of visible and hidden vectors.

The joint distribution is known as the Boltzmann Distribution which gives the probability that a particle can be observed in the state with the energy E. Unfortunately it is very difficult to calculate the joint probability due to the huge number of possible combination of v and h in the partition function Z. Much easier is the calculation of the conditional probabilities of state h given the state v and conditional probabilities of state v given the state h:
<img src="figure5.png" width="200" height="70" title="Layers in RBM">

Eq. 3. Conditional probabilities for h and v.
It should be noticed beforehand (before demonstrating this fact on practical example) that each neuron in a RBM can only exist in a binary state of 0 or 1. The most interesting factor is the probability that a hidden or visible layer neuron is in the state 1 — hence activated. 

# Contrastive Divergence

### Gibbs Sampling
The first part of the training is called Gibbs Sampling. Given an input vector v we are using p(h|v) for prediction of the hidden values h via sampling. Knowing the hidden values we use p(v|h) for prediction of new input values v via sampling. This process is repeated k times. After k iterations, we obtain the visible vector $v_k$ which was recreated from original input values $v_0$.
<img src="figure8.png" width="500" height="300" title="Layers in RBM">

The gibbs function *gibbs* is divided into subparts: <br>
1.*sampleHiddenLayer * <br>
2.*sampleVisibleLayer*

Let's look at *sampleHiddenLayer* now.

### Sample Hidden Layer

You already know that given an input vector v the probability for a single hidden neuron j being activated is:
<img src="figure6.png" width="400" height="200" title="Layers in RBM">

Eq. 4
Here is σ the Sigmoid function.

*sampleHiddenLayer* takes the visible layer as input to calculate the hidden layer using Eq. 4 *h1Pdf* and then samples it to get * h1_sample*

    v_sample: given visible layer matrix; matrix because a batch of data points will be trained at one go
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    hPdf: distribution of hidden layer; a matrix for batch of datapoints = p(h|v)
    h_sample: sampled hidden layer matrix

In [12]:
#graded
def sampleHiddenLayer(v_sample):
    
    # write the code for calculation of hPdf using vectorized implementation of Eq 4
    hPdf = sigmoid(np.add(np.dot(v_sample,W),hiddenBias)) # ~ 1 line
    
    # Here, np.random.binomial is used to create the hidden layer sample matrix
    h_sample = np_rng.binomial(size=hPdf.shape, n=1, p=hPdf)
#     print("W[0]:",W[0])
#     print("hiddenBias:",hiddenBias)
#     print('h_sample.shape',h_sample.shape)
#     print('v_sample.shape',v_sample.shape)
    
    return [hPdf, h_sample]

### Sample Visible Layer
Similarly, the probability that a binary state of a visible neuron i is set to 1 is:
<img src="figure7.png" width="400" height="200" title="Layers in RBM">
Eq. 5

As seen in equation 5, we will be writing a function to sample the Visible Layer.
This function samples the visible layer based on the sampled data of hidden layer. <br>

There are some differences in writing the function *sampleVisibleLayer*. <br>Firstly, we use np.random.multinomial to sample the visible layer *v_sample* from the distribution *vPdf*. <br>Secondly,elements of *vPdf* needs to sum to 1 as the function np.random.multinomial used to sample the visible layer takes on probability distributions as *pvals*. In other words, you are finding the softmax values. <br> Thirdly, we also make use of the *D* to sample the visible layer as each document has different word count.
    
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    vPdf: distribution of visible layer; a matrix for batch of datapoints = p(v|h)
    v_sample: sampled visible layer matrix
    

In [13]:
#graded
def sampleVisibleLayer(h_sample, D):
    
    # complete the following function such that vPdf has the sum of entries equal to 1 for each of the datapoints in the batch
    # you have to use axis = 1 in writing the denominator
    numerator = sigmoid(np.add(np.dot(h_sample,W.T),visibleBias))# ~1 line

    denominator = numerator.sum(axis=1)# ~1 line
    vPdf = (numerator/denominator[:,None]) #.flatten() # ~1 line
    
#     print('numerator.shape',numerator.shape)
#     print('h_sample.shape',h_sample.shape)
#     print('W.T.shape',W.T.shape)
#     print('denominator.shape',denominator.shape)
    
    # Here np.random.multinomial is used to sample as each document has different number of words 
    # and hence D is also a parameter in sampling
    v_sample = np.zeros((batchSize, vPdf.shape[1]))
    for i in range(batchSize):
        v_sample[i] = np_rng.multinomial(size=1, n=D[i], pvals=vPdf[i])
    return [vPdf, v_sample]

### Sampling

We use the above functions to write the function *gibbs* to run one iteration of gibbs sampling. Note that we are calculating the visible layer samples first and then using it to calculate he hidden layer sample. It'll become clear soon why we are doing so when you write the function for Contrastive Divergence.
    
    Input:
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    Output:
    vPdf: distribution of visible layer
    v_sample: sampled visible layer matrix
    hPdf: distribution of hidden layer
    h_sample: sampled hidden layer matrix

In [14]:
#graded
def gibbs(h_sample, D):
    
    #use sampleVIsibleLayer and sampleHiddenLayer 
    vPdf, v_sample = sampleVisibleLayer(h_sample, D) # ~1 line
    hPdf, h_sample = sampleHiddenLayer(v_sample) # ~1 line
    return [vPdf, v_sample, hPdf, h_sample ]


### Contrastive Divergence

You have learned that Contrastive Divergence updates weights after one iteration of Gibbs Sampling. Here, we shall perform *k* such iterations in Contrastive Divergence. 
The update of the weight matrix happens post the Contrastive Divergence step. 


Now, we will writing a funtion to run the contrastive divergence algorithm in k steps
    
    Input:
    data: batch data (visible layer)
    k: no of iterations for gibbs sampling
    
    Output:
    hiddenPDF_data: distribution of the hidden layer based on data
    visibleSamples: visible samples generated by gibbs sampling 
    hiddenPDF: distribution of the hidden layer based on samples generated by gibbs

In [15]:
#graded
def cd_k(data,k):
    
    D = data.sum(axis=1)
    hiddenPDF_data, hiddenSample_data = sampleHiddenLayer(data) # sample the hidden layer using the input data
    chain_start = hiddenSample_data

    for step in range(k):
        if step == 0:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples  = gibbs(chain_start,D) # perform gibbs sampling using chain_start
        else:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples = gibbs(hiddenSamples,D) # perform gibbs sampling using hiddenSamples
    return hiddenPDF_data, visibleSamples, hiddenPDF


## Training the Model

Vectors $v_0$ and $v_k$ are used to calculate the activation probabilities for hidden layers $p(h_0|v_0)$ and $p(h_k|v_k)$ (Eq.4). The difference between the outer products of those probabilities with input vectors $v_0$ and $v_k$ results in the update matrix:
<img src="figure9.png" width="300" height="200" title="Layers in RBM">
Eq. 6. Update matrix. **Figure out** the vectorized implementation for this.

In order to calculate $\Delta (bias)$, <br>
<center>$\Delta (visiblebias) = average\_ across\_ batch(v_0 - v_k)$ </center> 
<center>$\Delta (hiddenbias) = average\_across\_ batch(p(h_0|v_0) - p(h_k|v_k))$ </center> 

Using the update matrix the new weights can be calculated with momentum gradient ascent, given by:
<center>  $mW_t = \gamma \ mW_{t-1} - \Delta W$</center> 
<center>  $mvisiblebias_t = \gamma \ mvisiblebias_{t-1} - \Delta visiblebias$</center>
<center>  $mhiddenbias_t = \gamma \ mhiddenbias_{t-1} - \Delta hiddenbias$</center><br>
<center>  $W_t = W_{t-1} + \alpha \ mW_t$</center> 
<center>  $visiblebias_t = visiblebias_{t-1} + \alpha \ mvisiblebias_t$</center> 
<center>  $hiddenbias_t = hiddenbias_{t-1} + \alpha \ mhiddenbias_t$</center> 


Eq. 7. Update rule for the weights.

Note that in the code implementation below <br>
 hiddenPDF_data is $p(h_0|v_0)$ <br>
 visibleSamples is $v_k$ <br>
 hiddenPDF is $p(h_k|v_k)$ <br>
 mdata is $v_0$ <br>
 eta is $\alpha$ <br>
 mrate is $\gamma$ <br>
These will be helpful in writing the weight updates.

In this we will write a function which iterates over our data for *epochs*.
At every epoch we shuffle the data and then run CD on a mini batch size defined by *batchSize*

### Initializations

>**Note**: I have experimented with momentum coeff, learning rate, epoch & no. of gibbs sampling interations. The results of these experiments is shown in the end of the notebook. **Please do not run the notebook at once as these may get over-written**. 

In [16]:
"""
visibleUnits: no of words in your Bag of words Model
hiddenUnits: no of topics
batchSize: data slice to be selected 
epochs: no of iterations
eta: learning rate
mrate: momentum coefficient
W : weights between the visible and hidden layer
visibleBias, hiddenBias: biases for visible and hidden layer respectively
"""

# define batch size
batchSize = 200

epochs = 10 
eta = 0.001 
mrate = 0.5

# initialise weights
weightinit=0.01
W = weightinit * np_rng.randn(visibleUnits, hiddenUnits)
visibleBias = weightinit * np_rng.randn(visibleUnits)
hiddenBias = np.zeros((hiddenUnits))

# number of interations of gibbs sampling
k=500 


'\nvisibleUnits: no of words in your Bag of words Model\nhiddenUnits: no of topics\nbatchSize: data slice to be selected \nepochs: no of iterations\neta: learning rate\nmrate: momentum coefficient\nW : weights between the visible and hidden layer\nvisibleBias, hiddenBias: biases for visible and hidden layer respectively\n'

### Training

>**Note**: I experimented with a few different ways to initialize momentum weights & biases and after some experimentation I went with the same initialization logic as the weight & bias for visible and hidden layers.

In [17]:
#graded
def train(dataX,k):
    
    mW = 0.1 * np_rng.randn(visibleUnits, hiddenUnits)  # initialise momentum_weights
    mvisibleBias =  np_rng.randn(visibleUnits)  # initialise momentum_visiblebiases
    mhiddenBias =  np.zeros((hiddenUnits)) #hiddenBias # initialise momentum_hiddenbiases
    global W,visibleBias,hiddenBias,mrate,batchSize,epochs # calling the variables initialized at the beginning
#     print("initial hiddenBias:", hiddenBias)
#     print("initial hiddenBias[0]:", W[0])
    for epoch in range(epochs):
        epoch_error.append(np.average(np.abs(eta * W)))
        np_rng.shuffle(dataX) #shuffling the data
        
        for i in range(0, dataX.shape[0], batchSize):
            mData = dataX[i:i+batchSize] #select a batch of datapoints
            hiddenPDF_data, visibleSamples, hiddenPDF = cd_k(mData,k) # perfrom Contrastive Divergence on the batch for k iterations

            mW = mrate*mW - (np.dot(mData.T,hiddenPDF_data) - np.dot(visibleSamples.T,hiddenPDF)) #write the momentum update equation for weight matrix
            mvisibleBias = mrate*mvisibleBias - np.mean(mData-visibleSamples,axis=0) #write the momentum update equation for visiblebias vector
            mhiddenBias = mrate*mhiddenBias - np.mean(hiddenPDF_data-hiddenPDF,axis=0) #write the momentum update equation for hiddenbias vector

            W =  W + eta*mW#weight update equation
            visibleBias =  visibleBias + eta*mvisibleBias #visible bias update equation
            hiddenBias =  hiddenBias + eta*mhiddenBias #hidden bias update equation
        
#     print('W')
#     print(W.T[0][:20])
#     print(W.T[1][:20])
#     print(W.T[4][:20])

## Train the Model

This will take around 10 minutes of time to run.

In [ ]:
train(trainX.toarray(),k)

# Word Distribution based on Topics
In this function, we are finding the distribution of words over the topics. You can take a look at the words under each topic and see what they are talking about. The number of topics is the number of neurons in the hidden layer. <br>
<br>

For each topic, the function prints the top 15 words that describe the topic. You can see that some of the words occur in multiple topics.

    topic: number of hidden layers
    voc: indexing of the vocabulary
    
Feel free to change the number of iterations of gibbs sampling in Contrastive Divergence and see how the distribution of words change under the topics.

In [22]:
def worddist( topic, voc):
    
    """
    Initialize every topic =1 once 
    """
    vecTopics = np.zeros((topic, topic))
    for i in range(len(vecTopics)):
        vecTopics[i][i] = 1
    
    
    for i, vecTopic in enumerate(vecTopics):
       
        numerator = np.exp(np.dot(vecTopic, W.T) + visibleBias) #extract weights for current topic only and add bias
        denominator = numerator.sum().reshape((1, 1))
        word_distribution = (numerator/denominator).flatten() 
#         print('numerator.shape',numerator.shape)        
#         print('word_distribution.shape',word_distribution.shape)
        
        tmpDict = {}
        for j in voc.keys():
            tmpDict[j] = word_distribution[voc[j]]
            
        print('topic', str(i), ':', vecTopic)
        lm=0
        for word, prob in sorted(tmpDict.items(), key=lambda x:x[1], reverse=True):
            print ( word, str(prob))
            lm+=1
            if lm==15:
                break
        print('\n')

>**Note**: Following are the final outcomes from some (not all) experiments with the hyperparameters. The outcomes below are words with the highest probability (top 15) assigned to words for ever topic present.

**Experiment 1** - Results with mrate=0.5,**eta=0.001,k=100**,epochs=10

In [23]:
worddist( hiddenUnits, tf.vocabulary_)

topic 0 : [1. 0. 0. 0. 0.]
spinn 0.0006524152040901103
controls 0.0006505715182136301
hs850 0.0006481419325434056
tone 0.0006447847169387012
numbers 0.0006439053608644768
regretted 0.0006436802948498797
shiny 0.0006435412148758001
speakerphone 0.0006431158154288662
flaws 0.0006430379730379628
drawback 0.0006424342772907553
loose 0.0006422844509337879
somewhat 0.0006419455676309631
activesync 0.0006415275000868625
dissapointed 0.0006410321691586049
usage 0.000640975200333985


topic 1 : [0. 1. 0. 0. 0.]
echo 0.0006543179137761169
speakerphone 0.0006477231559574738
controls 0.0006476400659267483
lately 0.0006457033011311156
cingulair 0.0006445616035771799
music 0.0006440163053673764
hs850 0.0006435972455827964
shiny 0.0006435881870193016
designed 0.0006434086709333896
europe 0.0006420768262223621
e715 0.0006417701890851023
noted 0.0006415291059249597
soon 0.0006415189969968092
3o 0.0006410932027072436
lesson 0.0006408920611860003


topic 2 : [0. 0. 1. 0. 0.]
shiny 0.0006505131777529542
h

**Experiment 2** - Results with mrate=0.5,eta=0.001,**k=500**,epochs=10

In [20]:
worddist( hiddenUnits, tf.vocabulary_)

topic 0 : [1. 0. 0. 0. 0.]
spinn 0.000649194443156827
amazing 0.0006483275572622738
regretted 0.0006441503106977802
controls 0.0006433145775703704
somewhat 0.0006424092617452834
heavy 0.0006424032828502251
gadgets 0.000642325677554717
majority 0.0006413770161594453
inform 0.0006412618917992938
slider 0.0006411853041204485
crappy 0.0006409221440915779
loose 0.0006408289262711453
exclaim 0.000640591031363337
numbers 0.0006398689911235407
specially 0.0006397041962328129


topic 1 : [0. 1. 0. 0. 0.]
echo 0.0006495883679500362
soon 0.0006459328032907043
muddy 0.0006444324684799446
e715 0.0006426542749020328
deffinitely 0.0006426377509450384
2mp 0.0006425245698774286
speakerphone 0.0006420619777497197
type 0.0006416737821420931
3o 0.0006405973497929723
friends 0.000640548152880026
controls 0.0006404701344259259
protective 0.0006398682801234707
died 0.0006396240543064109
regretted 0.000639539791951766
noted 0.0006392708815084472


topic 2 : [0. 0. 1. 0. 0.]
muddy 0.000652599876487995
needless

**Experiment 3** - Results with **mrate=0.8**,eta=0.001,k=500,epochs=10

In [20]:
worddist( hiddenUnits, tf.vocabulary_)

topic 0 : [1. 0. 0. 0. 0.]
gadgets 0.0006688188940664466
numbers 0.000664136210462444
receive 0.0006629136023917666
drawback 0.0006607478726347344
overly 0.0006605251868973922
steer 0.000659433033705906
heavy 0.0006590764352005376
hs850 0.0006589531213337341
arguing 0.0006583260343221787
enter 0.0006581631870000036
slider 0.0006576393472864829
contract 0.0006566068922049028
monkeys 0.0006566022802014712
flaws 0.0006565741862729577
palmtop 0.0006558410236302972


topic 1 : [0. 1. 0. 0. 0.]
receive 0.0006750350183489304
lately 0.0006653190596512278
overly 0.0006626604894263039
muddy 0.0006606113941250434
contract 0.0006600314182142065
5020 0.0006596677823794536
gadgets 0.0006596160503165852
grtting 0.0006595543419812437
colleague 0.0006587315200260943
steer 0.0006582334302108911
unacceptible 0.0006577013918603521
e715 0.0006567518056067941
mention 0.0006561431424203019
protective 0.0006554691048581436
designed 0.0006553676974423795


topic 2 : [0. 0. 1. 0. 0.]
receive 0.00067319687341394

**Experiment 4** - Results with mrate=0.5,eta=0.001,**k=1000**,epochs=10

In [20]:
worddist( hiddenUnits, tf.vocabulary_)

topic 0 : [1. 0. 0. 0. 0.]
spinn 0.000648626254842024
pain 0.0006449641568013654
loose 0.0006448808319196189
controls 0.0006438788868137451
regretted 0.000642668508680707
somewhat 0.0006423241281897415
slider 0.0006421113768316097
complaints 0.0006409632747041718
dissapointed 0.0006404544331099966
speakerphone 0.000640215154251622
scratched 0.000640188797354127
arguing 0.0006401763066359821
haven 0.0006399249148608437
hs850 0.0006396813236744327
severe 0.0006395825325553146


topic 1 : [0. 1. 0. 0. 0.]
speakerphone 0.0006449305698656856
cingulair 0.000643973263473468
type 0.0006437066564077254
pain 0.0006434861904794708
protective 0.0006431809723847077
echo 0.0006410558955848775
controls 0.000640993752613397
3o 0.0006409175138877158
generally 0.0006407268419302515
noted 0.0006407050199849747
sources 0.0006399400467966577
e715 0.0006396823777407392
wood 0.0006394703022148695
2mp 0.0006392565212680539
overly 0.0006390813463366422


topic 2 : [0. 0. 1. 0. 0.]
wood 0.0006440391304989238
ne

**Note**: All experiments that I ran are not shown above, this is only a subset.

>**Observation**: Experiment 3 far provided the best results in terms of placing similar words in the same topic. For example, topic 1 includes words like 'regretted', 'complaints', 'disappointed' associated to negative reviews & complaints; topic 2 includes words describing products such as 'speakerphone', 'echo', 'controls' and so on. 


>**Conclusion**: With the default parameters i.e. with low K, high epochs and relatively high learning rate, the top 15 words across  the top 5 words were mostly the same. However, increasing  no. of iterations of gibbs sampling (k) and even with few epochs and lower learning rate we get more optimal results where we can see different set of top 15 words in the 5 topics. The model can be improved further by tweaking the hyperparameters.